In [88]:
# Setting up env variables for cleaner command line commands.
import os

os.environ["KEY"]="tlt_encode"
os.environ["NUM_GPUS"]="1"


os.environ["LOCAL_PROJECT_DIR"] = "/home/latona/tashiro/taolearning"

os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)

os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "specs"
)

os.environ["LOCAL_NGC_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "ngccli","ngc-cli",
)



# container 内の それぞれの PATH を指定する
# 動かす必要なし
os.environ["CONTAINER_PROJECT_DIR"] = "/workspace/tao-experiments/"
os.environ["CONTAINER_DATA_DIR"] =    "/workspace/tao-experiments/data"
os.environ["CONTAINER_SPECS_DIR"] =   "/workspace/tao-experiments/specs"

In [89]:
# 環境変数確認

!echo "KEY = $KEY"
!echo "NUM_GPUS = $NUM_GPUS"
!echo ""
!echo "LOCAL_PROJECT_DIR = $LOCAL_PROJECT_DIR"
!echo "LOCAL_DATA_DIR    = $LOCAL_DATA_DIR"
!echo "LOCAL_SPECS_DIR   = $LOCAL_SPECS_DIR"
!echo ""
!echo "CONTAINER_PROJECT_DIR = $CONTAINER_PROJECT_DIR"
!echo "CONTAINER_DATA_DIR    = $CONTAINER_DATA_DIR"
!echo "CONTAINER_SPECS_DIR   = $CONTAINER_SPECS_DIR"
!echo ""
!echo "LOCAL_NGC_DIR = $LOCAL_NGC_DIR"
!echo ""



KEY = tlt_encode
NUM_GPUS = 1

LOCAL_PROJECT_DIR = /home/latona/tashiro/taolearning
LOCAL_DATA_DIR    = /home/latona/tashiro/taolearning/data
LOCAL_SPECS_DIR   = /home/latona/tashiro/taolearning/specs

CONTAINER_PROJECT_DIR = /workspace/tao-experiments/
CONTAINER_DATA_DIR    = /workspace/tao-experiments/data
CONTAINER_SPECS_DIR   = /workspace/tao-experiments/specs

LOCAL_NGC_DIR = /home/latona/tashiro/taolearning/ngccli/ngc-cli



In [91]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source":      os.environ["LOCAL_DATA_DIR"],
            "destination": os.environ["CONTAINER_DATA_DIR"],
        },
        # Mapping the specs directory.
        {
            "source":      os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["CONTAINER_SPECS_DIR"]
        },
        # NGC
        {
            "source":      os.environ["LOCAL_NGC_DIR"],
            "destination": "/opt/ngccli/ngc"
        }
        
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [92]:
# CHECK mount setting file exists
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/latona/tashiro/taolearning/data",
            "destination": "/workspace/tao-experiments/data"
        },
        {
            "source": "/home/latona/tashiro/taolearning/specs",
            "destination": "/workspace/tao-experiments/specs"
        },
        {
            "source": "/home/latona/tashiro/taolearning/ngccli/ngc-cli",
            "destination": "/opt/ngccli/ngc"
        }
    ]
}

In [14]:
!python3 -m pip install nvidia-pyindex
!python3 -m pip install nvidia-tao

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-cp36-none-any.whl size=8401 sha256=dfcf1d072808df4d5f434f47d696921df2554c14203f9f0b4ffd3f63e31a83da
  Stored in directory: /tmp/pip-ephem-wheel-cache-hzo0ztgb/wheels/3d/0b/07/1f740912ad4b0f37d363faec6ded960167cae063c9b3c55722
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 153kB 31.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 25.7MB/s eta 0:00:01


In [15]:
# View the versions of the TAO launcher
!tao info
!python3 -V

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 2.0
toolkit_version: 3.22.05
published_date: 05/25/2022
Python 3.6.9 :: Anaconda, Inc.


In [101]:
print("TFrecords conversion spec file for kitti training")
!cat $LOCAL_SPECS_DIR/detectnet_tfrecords.txt
print()
print("!!!!!!!!!!!!!!!")
print("root_directory_path should be", os.environ["CONTAINER_DATA_DIR"] + "/training")
print("")
print("these dir name should be")
!ls $LOCAL_DATA_DIR/training

TFrecords conversion spec file for kitti training
kitti_config {
  root_directory_path: "/workspace/tao-experiments/data/training"
  image_dir_name: "image_2"
  label_dir_name: "label_2"
  image_extension: ".jpg"
  partition_mode: "random"
  num_partitions: 2
  val_split: 14
  num_shards: 10
}
image_directory_path: "/workspace/tao-experiments/data/training"

!!!!!!!!!!!!!!!
root_directory_path should be /workspace/tao-experiments/data/training

these dir name should be
image_2  label_2


In [102]:
# Creating a new directory for the output tfrecords dump.
print("Converting Tfrecords for kitti trainval dataset")
!mkdir -p $LOCAL_DATA_DIR/tfrecords &&sudo rm -rf $LOCAL_DATA_DIR/tfrecords/*
!echo detectnet_v2 dataset_convert \
                  -d $CONTAINER_SPECS_DIR/detectnet_tfrecords.txt \
                  -o $CONTAINER_SPECS_DIR/tfrecords/kitti_trainval

Converting Tfrecords for kitti trainval dataset
detectnet_v2 dataset_convert -d /workspace/tao-experiments/specs/detectnet_tfrecords.txt -o /workspace/tao-experiments/specs/tfrecords/kitti_trainval


In [95]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/

合計 564
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 root root  7380  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 root root 11070  6月 23 16:18 kitti_trainval-fold-000-of-002-shard-00009-of-00010
-rw-r--r-- 1 root root 47968  6月 23 16:18 kitti_train

In [17]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-06-23 13:34:16--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
ngc.nvidia.com (ngc.nvidia.com) をDNSに問いあわせています... 143.204.86.23, 143.204.86.69, 143.204.86.81, ...
ngc.nvidia.com (ngc.nvidia.com)|143.204.86.23|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 33925833 (32M) [application/zip]
`/home/latona/tashiro/taolearning/ngccli/ngccli_cat_linux.zip' に保存中

ngccli_cat_linux.zi 100%[===================>]  32.35M  91.1MB/s    in 0.4s    

2022-06-23 13:34:16 (91.1 MB/s) - `/home/latona/tashiro/taolearning/ngccli/ngccli_cat_linux.zip' へ保存完了 [33925833/33925833]

Archive:  /home/latona/tashiro/taolearning/ngccli/ngccli_cat_linux.zip
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/multidict/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/multidict/_multidict.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngcc

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/unicodedata.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_codecs_cn.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_json.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/grp.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_posixshmem.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_bisect.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_codecs_hk.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/_blake2.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/lib-dynload/pyexpat

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/sagemaker/2017-07-24/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/sagemaker/2017-07-24/waiters-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/sagemaker/2017-07-24/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/budgets/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/budgets/2016-10-20/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/budgets/2016-10-20/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/budgets/2016-10-20/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/budgets/2016-10-20/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudsearchdomain/
   creating: /home/latona/tashiro/taolearning/ngcc

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2015-04-15/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2015-04-15/waiters-2.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-04-01/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-04-01/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-04-01/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-04-01/waiters-2.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-09-15/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-09-15/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/ec2/2016-09-15/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/bo

   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/codestar-connections/2019-12-01/
 extracting: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/codestar-connections/2019-12-01/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/codestar-connections/2019-12-01/service-2.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/honeycode/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/honeycode/2020-03-01/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/honeycode/2020-03-01/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/honeycode/2020-03-01/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/honeycode/2020-03-01/paginators-1.sdk-extras.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cognito-identity/
   creati

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/dynamodb/2012-08-10/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/dynamodb/2012-08-10/waiters-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/dynamodb/2012-08-10/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/transcribe/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/transcribe/2017-10-26/
 extracting: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/transcribe/2017-10-26/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/transcribe/2017-10-26/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/transcribe/2017-10-26/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/acm-pca/
   creating: /home/latona/tashiro/taolearning/ng

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2016-11-25/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2018-11-05/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2018-11-05/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2018-11-05/service-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2018-11-05/waiters-2.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2018-11-05/examples-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2015-09-17/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2015-09-17/paginators-1.json  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/botocore/data/cloudfront/2015-09-17/service-2.json 

  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/grpc/_cython/cygrpc.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/libffi-9c61262e.so.8.1.0  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/examples/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/examples/s3.rst  
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/examples/cloudfront.rst  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/data/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/data/iam/
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/data/iam/2010-05-08/
  inflating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/data/iam/2010-05-08/resources-1.json  
   creating: /home/latona/tashiro/taolearning/ngccli/ngc-cli/boto3/data/sns/
   creating: /home/latona/tashiro/taolearning/ngccli/n

In [96]:
# List models available in the model registry.
!./ngccli/ngc-cli/ngc registry model list nvidia/tao/pretrained_detectnet_v2:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| vgg19 | 82.6  | 80    | 1     | V100  | 153.8 | 153.7 | UPLOA | Aug   |
|       |       |       |       |       |       | 7 MB  | D_COM | 24,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| vgg16 | 82.2  | 80    | 1     | V100  | 113.2 | 113.2 | UPLOA | Aug   |
|       |       |       |       |       |       | MB    | D_COM | 24,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| squee | 65.67 | 80    | 1     | V100  | 6.5   | 6.46  | UPLOA | Aug   |
| zenet |       |       |       |     

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models

*Note: The training may take hours to complete. Also, the remaining notebook, assumes that the training was done in single-GPU mode. When run in multi-GPU mode, please expect to update the pruning and inference steps with new pruning thresholds and updated parameters in the clusterfile.json accordingly for optimum performance.*

*Detectnet_v2 now supports restart from checkpoint. In case the training job is killed prematurely, you may resume training from the closest checkpoint by simply re-running the **same** command line. Please do make sure to use the <u>**same number of GPUs**</u> when restarting the training.*

*When running the training with NUM_GPUs>1, you may need to modify the `batch_size_per_gpu` and `learning_rate` to get similar mAP as a 1GPU training run. In most cases, scaling down the batch-size by a factor of NUM_GPU's or scaling up the learning rate by a factor of NUM_GPU's would be a good place to start.* 

In [100]:
!echo  detectnet_v2 train -e $CONTAINER_SPECS_DIR/detectnet_train.txt \
                        -r $CONTAINER_DATA_DIR/experiment_dir_unpruned \
                        -k $KEY \
                        -n resnet18_detector \
                        --gpus $NUM_GPUS

detectnet_v2 train -e /workspace/tao-experiments/specs/detectnet_train.txt -r /workspace/tao-experiments/data/experiment_dir_unpruned -k tlt_encode -n resnet18_detector --gpus 1


In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights

## 5. Evaluate the trained model <a class="anchor" id="head-5"></a>

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_train_resnet18_kitti.txt\
                           -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                           -k $KEY

## 6. Prune the trained model <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Applicable for resnets and mobilenets`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

*Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold to use is dependent on the dataset. A pth value `5.2e-6` is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.*

*For some internal studies, we have noticed that a pth value of 0.01 is a good starting point for detectnet_v2 models.*

In [ ]:
# Create an output directory if it doesn't exist.
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao detectnet_v2 prune \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/resnet18_detector.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/resnet18_nopool_bn_detectnet_v2_pruned.tlt \
                  -eq union \
                  -pth 0.0000052 \
                  -k $KEY

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Retrain the pruned model <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification with pretrained weights as pruned model.

*Note: For retraining, please set the `load_graph` option to `true` in the model_config to load the pruned model graph. Also, if after retraining, the model shows some decrease in mAP, it could be that the originally trained model was pruned a little too much. Please try reducing the pruning threshold (thereby reducing the pruning ratio) and use the new model to retrain.*

*Note: DetectNet_v2 now supports Quantization Aware Training, to help with optmizing the model. By default, the training in the cell below doesn't run the model with QAT enabled. For information on training a model with QAT, please refer to the cells under [section 11](#head-11)*

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to include the 
# newly pruned model as a pretrained weights and, the
# load_graph option is set to true 
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain \
                        -k $KEY \
                        -n resnet18_detector_pruned \
                        --gpus $NUM_GPUS

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

## 8. Evaluate the retrained model <a class="anchor" id="head-8"></a>

This section evaluates the pruned and retrained model, using the `evaluate` command.

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                           -k $KEY

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `inference` tool to generate inferences on the trained models. To render bboxes from more classes, please edit the spec file `detectnet_v2_inference_kitti_tlt.txt` to include all the classes you would like to visualize and edit the rest of the file accordingly.

In [ ]:
# Running inference for detection on n images
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_tlt.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

The `inference` tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/tlt_infer_testing/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/tlt_infer_testing/labels`

*Note: To run inferences for a single image, simply replace the path to the -i flag in `inference` command with the path to the image.*

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the first 12 images.
OUTPUT_PATH = 'tlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export <a class="anchor" id="head-10"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_final
# Removing a pre-existing copy of the etlt if there has been any.
import os
output_file=os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'],
                         "experiment_dir_final/resnet18_detector.etlt")
if os.path.exists(output_file):
    os.system("rm {}".format(output_file))
!tao detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                  -k $KEY

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/experiment_dir_final

### A. Int8 Optimization <a class="anchor" id="head-10-1"></a>
DetectNet_v2 model supports int8 inference mode in TensorRT. 
In order to use int8 mode, we must calibrate the model to run 8-bit inferences -

* Generate calibration tensorfile from the training data using detectnet_v2 calibration_tensorfile
* Use tao <task> export to generate int8 calibration table.

*Note: For this example, we generate a calibration tensorfile containing 10 batches of training data.
Ideally, it is best to use at least 10-20% of the training data to do so. The more data provided during calibration, the closer int8 inferences are to fp32 inferences.*

*Note: If the model was trained with QAT nodes available, please refrain from using the post training int8 optimization as mentioned below. Please export the model in int8 mode (using the arg `--data_type int8`) with just the path to the calibration cache file (using the argument `--cal_cache_file`)*

In [ ]:
!tao detectnet_v2 calibration_tensorfile -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti.txt \
                                         -m 10 \
                                         -o $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.tensor

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration.bin
!tao detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/resnet18_detector_pruned.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                  -k $KEY  \
                  --cal_data_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.tensor \
                  --data_type int8 \
                  --batches 10 \
                  --batch_size 4 \
                  --max_batch_size 4\
                  --engine_file $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.bin \
                  --verbose

### B. Generate TensorRT engine <a class="anchor" id="head-10-2"></a>
Verify engine generation using the `tao-converter` utility included with the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
!tao converter $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.etlt \
                   -k $KEY \
                    -c $USER_EXPERIMENT_DIR/experiment_dir_final/calibration.bin \
                   -o output_cov/Sigmoid,output_bbox/BiasAdd \
                   -d 3,384,1248 \
                   -i nchw \
                   -m 64 \
                   -t int8 \
                   -e $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector.trt \
                   -b 4

## 11. Verify Deployed Model <a class="anchor" id="head-11"></a>
Verify the exported model by visualizing inferences on TensorRT.
In addition to running inference on a `.tlt` model in [step 9](#head-9), the `inference` tool is also capable of consuming the converted `TensorRT engine` from [step 10.B](#head-10-2).

*If after int-8 calibration the accuracy of the int-8 inferences seem to degrade, it could be because the there wasn't enough data in the calibration tensorfile used to calibrate thee model or, the training data is not entirely representative of your test images, and the calibration maybe incorrect. Therefore, you may either regenerate the calibration tensorfile with more batches of the training data and recalibrate the model, or calibrate the model on a few images from the test set. This may be done using `--cal_image_dir` flag in the `export` tool. For more information, please follow the instructions in the USER GUIDE.

### A. Inference using TensorRT engine <a class="anchor" id="head-11-1"></a>

In [ ]:
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt.txt \
                            -o $USER_EXPERIMENT_DIR/etlt_infer_testing \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'etlt_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 11. QAT workflow <a class="anchor" id="head-12"></a>
This section delves into the newly enabled Quantization Aware Training feature with DetectNet_v2. The workflow defined below converts a pruned model from section [5](#head-5) to enable QAT and retrain this model to while accounting the noise introduced due to quantization in the forward pass. 

### A. Convert pruned model to QAT and retrain <a class="anchor" id="head-12-1"></a>
All detectnet models, unpruned and pruned models can be converted to QAT models by setting the `enable_qat` parameter in the `training_config` component of the spec file to `true`.

In [ ]:
# Printing the retrain experiment file. 
# Note: We have updated the experiment file to convert the
# pretrained model to qat mode by setting the enable_qat
# parameter.
!cat $LOCAL_SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt

In [ ]:
!tao detectnet_v2 train -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                        -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat \
                        -k $KEY \
                        -n resnet18_detector_pruned_qat \
                        --gpus $NUM_GPUS

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

### B. Evaluate QAT converted model <a class="anchor" id="head-12-2"></a>
This section evaluates a QAT enabled pruned retrained model. The mAP of this model should be comparable to that of the pruned retrained model without QAT. However, due to quantization, it is possible sometimes to see a drop in the mAP value for certain datasets.

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                           -k $KEY \
                           -f tlt

### C. Export QAT trained model to int8 <a class="anchor" id="head-12-3"></a>
Export a QAT trained model to TensorRT parsable model. This command generates an .etlt file from the trained model and the serializes corresponding int8 scales as a TRT readable calibration cache file.

In [ ]:
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt
!rm -rf $LOCAL_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin
!tao detectnet_v2 export \
                  -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/resnet18_detector_pruned_qat.tlt \
                  -o $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.etlt \
                  -k $KEY  \
                  --data_type int8 \
                  --batch_size 64 \
                  --max_batch_size 64\
                  --engine_file $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                  --cal_cache_file $USER_EXPERIMENT_DIR/experiment_dir_final/calibration_qat.bin \
                  --verbose

### D. Evaluate a QAT trained model using the exported TensorRT engine <a class="anchor" id="head-12-4"></a>
This section evaluates a QAT enabled pruned retrained model using the TensorRT int8 engine that was exported in [Section C](#head-12-3). Please note that there maybe a slight difference (~0.1-0.5%) in the mAP from [Section B](#head-12-2), oweing to some differences in the implementation of quantization in TensorRT.

*Note: The TensorRT evaluator might be slightly slower than the TAO evaluator here, because the evaluation dataloader is pinned to the CPU to avoid any clashes between TensorRT and TAO instances in the GPU. Please note that this tool was not intended and has not been developed for profiling the model. It is just a means to qualitatively analyse the model.*

*Please use native TensorRT or DeepStream for the most optimized inferences.*

In [ ]:
!tao detectnet_v2 evaluate -e $SPECS_DIR/detectnet_v2_retrain_resnet18_kitti_qat.txt \
                           -m $USER_EXPERIMENT_DIR/experiment_dir_final/resnet18_detector_qat.trt.int8 \
                           -f tensorrt

### E. Inference using QAT engine <a class="anchor" id="head-12-5"></a>
Run inference and visualize detections on test images, using the exported TensorRT engine from [Section C](#head-12-3).

In [ ]:
!tao detectnet_v2 inference -e $SPECS_DIR/detectnet_v2_inference_kitti_etlt_qat.txt \
                            -o $USER_EXPERIMENT_DIR/tlt_infer_testing_qat \
                            -i $DATA_DOWNLOAD_DIR/testing/image_2 \
                            -k $KEY

In [ ]:
# visualize the first 12 inferenced images.
OUTPUT_PATH = 'tlt_infer_testing_qat/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)